In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 研究区
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
# roi = ee.Geometry.Rectangle([89.8565, 34.6503,91.4664, 35.8282])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

# 加载sentinel-2图像

In [4]:
# 加载sentinel 图像
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)
s2 = ee.Image('COPERNICUS/S2_SR/20200209T025851_20200209T030437_T50RKU')
Map.addLayer(maskS2clouds(s2.clip(roi)),{'min': 0.0,'max': 0.3,'bands': ['B8', 'B4', 'B3']},'S2')

# 加载Landsat8图像

In [5]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 图像可视化参数
visParams = {'bands': ['B5', 'B4', 'B3'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})

# 移除山地阴影
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)# .reproject('EPSG:3857',None,30)
def removeShadow(image):
    azimuth = image.get('SOLAR_AZIMUTH_ANGLE')
    zenith = image.get('SOLAR_ZENITH_ANGLE')
    return image.updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True))

In [6]:
image_id = 'LC08_123039_20200209'
id = 'LANDSAT/LC08/C01/T1_SR/' + image_id
image2 = water_index(ee.Image(id))

# Map.addLayer(image2,visParams,'image2')
Map.addLayer(maskL8sr(image2),visParams,'mask image2')

# 加载样本数据

## 无季节性样本

In [7]:
## 导入采集的样本集
samples_season = ee.FeatureCollection('users/311605001111/wuhan_new2020_noseason')
# print(samples_season.first().getInfo())
print('total sample number:{}'.format(samples_season.size().getInfo()))
a = samples_season.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples_season.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

total sample number:959
水体样本的数目： 357
陆地样本的数目： 602


## 全部样本

In [8]:
## 导入采集的样本集
samples = ee.FeatureCollection('users/311605001111/wuhan_new2020')
# print(samples.first().getInfo())
print('total sample number:{}'.format(samples.size().getInfo()))
a = samples.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = samples.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

total sample number:1529
水体样本的数目： 553
陆地样本的数目： 976


# k-mean均值聚类，样本过滤

In [9]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample

In [10]:
filtered_samples = k_mean(samples)
print("筛选后，剩余的样本数： ",filtered_samples.size().getInfo())
a = filtered_samples.filter(ee.Filter.eq('waterclass',1))
print('水体样本的数目：',a.size().getInfo())
a = filtered_samples.filter(ee.Filter.eq('waterclass',0))
print('陆地样本的数目：',a.size().getInfo())

筛选后，剩余的样本数：  1448
水体样本的数目： 550
陆地样本的数目： 898


# 训练RF训练器

In [12]:
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# 分类标签
label = 'waterclass'

In [13]:
trainedClassifier_noseason = ee.Classifier.smileRandomForest(20).train(samples_season,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

image_noseason = maskL8sr(image2).select(bands).classify(trainedClassifier_noseason).eq(1).rename('waterclass')
Map.addLayer(image_noseason.selfMask(),{'palette':['green']},'image_noseason')

## 过滤后

In [14]:
post_trainedClassifier = ee.Classifier.smileRandomForest(20).train(filtered_samples,label,bands)
post_image = maskL8sr(image2).select(bands).classify(post_trainedClassifier).eq(1).rename('waterclass')
Map.addLayer(post_image.selfMask(),{'palette':['yellow']},'post_image')

## 过滤前

In [15]:
pro_trainedClassifier = ee.Classifier.smileRandomForest(20).train(samples,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

pro_image = maskL8sr(image2).select(bands).classify(pro_trainedClassifier).eq(1).rename('waterclass')
Map.addLayer(pro_image.selfMask(),{'palette':['blue']},'pro_image')